In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install quandl

  Running setup.py bdist_wheel for inflection ... - done
  Stored in directory: /root/.cache/pip/wheels/9f/5a/d3/6fc3bf6516d2a3eb7e18f9f28b472110b59325f3f258fe9211
Successfully built inflection


In [0]:
import quandl
import pandas as pd
from collections import deque
import random
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt 
import keras
from keras.models import Sequential,load_model
from keras.layers import Dense,Dropout,LSTM,BatchNormalization,CuDNNLSTM,TimeDistributed
from keras.callbacks import TensorBoard,ModelCheckpoint
from sklearn.externals import joblib

Using TensorFlow backend.


In [0]:
SEQ_LEN=10
FUTURE_PERIOD_PREDICT=1
RATIO_TO_PREDICT=""
EPOCHS=10
BATCH_SIZE=32

In [0]:
file=open("drive/My Drive/stock_pred/nasdaq_100.txt")
file.seek(0)
company=[]
for i in range(7,len(file.readlines())):
    file.seek(0)
    temp=file.readlines()[i]
    r=''
    for j in temp:
        if j=='\t':
            break
        r+=j 
    company.append(r)    
    #print(r)
print(company)

['AAL', 'AAPL', 'ADBE', 'ADI', 'ADP', 'ADSK', 'ALGN', 'ALXN', 'AMAT', 'AMGN', 'AMZN', 'ATVI', 'ASML', 'AVGO', 'BIDU', 'BIIB', 'BMRN', 'CDNS', 'CELG', 'CERN', 'CHKP', 'CHTR', 'CTRP', 'CTAS', 'CSCO', 'CTXS', 'CMCSA', 'COST', 'CSX', 'CTSH', 'DISH', 'DLTR', 'EA', 'EBAY', 'ESRX', 'EXPE', 'FAST', 'FB', 'FISV', 'FOX', 'FOXA', 'GILD', 'GOOG', 'GOOGL', 'HAS', 'HSIC', 'HOLX', 'ILMN', 'INCY', 'INTC', 'INTU', 'ISRG', 'IDXX', 'JBHT', 'JD', 'KLAC', 'KHC', 'LBTYA', 'LBTYK', 'LRCX', 'MELI', 'MAR', 'MCHP', 'MDLZ', 'MNST', 'MSFT', 'MU', 'MXIM', 'MYL', 'NFLX', 'NTES', 'NVDA', 'NXPI', 'ORLY', 'PAYX', 'PCAR', 'BKNG', 'PYPL', 'QCOM', 'QRTEA', 'REGN', 'ROST', 'STX', 'SHPG', 'SIRI', 'SWKS', 'SBUX', 'SYMC', 'SNPS', 'TTWO', 'TSLA', 'TXN', 'TMUS', 'ULTA', 'VOD', 'VRTX', 'WBA', 'WDC', 'WDAY', 'VRSK', 'WYNN', 'XEL', 'XLNX']


In [0]:
quandl.ApiConfig.api_key = "23KLyzjn5UvKQog-DZyM"

In [0]:
data_st = quandl.get_table('WIKI/PRICES', ticker=company[2],
                         qopts={'columns': ['date', \
                                            'adj_close','adj_volume']},
                         date={'gte': '2003-01-20', 'lte': '2018-12-09'},
                         paginate=True)
print(data_st)
data_st.to_csv(f"drive/My Drive/stock_pred/data/{company[0]}.csv")

In [0]:
for i in company:
    data_st = pd.DataFrame()
    data_st = quandl.get_table('WIKI/PRICES', ticker=i,
                         qopts={'columns': ['date', \
                                            'adj_close','adj_volume']},
                         date={'gte': '2003-01-20', 'lte': '2018-12-08'},
                         paginate=True)
    data_st.to_csv(f"drive/My Drive/stock_pred/data/{i}.csv")


data_nas=quandl.get("NASDAQOMX/NDX")
data_nas.to_csv(f"drive/My Drive/stock_pred/data/nasdaq.csv")

In [0]:
df=pd.DataFrame()
df=pd.read_csv("drive/My Drive/stock_pred/data/nasdaq.csv")
df.rename(columns={"Trade Date":"date"},inplace=True)
#print(df)
df=df[["date","Index Value"]]
df.set_index("date",inplace=True)
print(df)

In [0]:
def preprocess_df(df1,df2,company):
        df1.dropna(inplace=True)
        df2.dropna(inplace=True)
        
        df1=df1.iloc[::-1]
        df2=df2.iloc[::-1]
        
        scaler=preprocessing.MinMaxScaler()
        scaler.fit(df1["adj_close"].values.reshape(-1,1))
        df1["adj_close"]=scaler.transform(df1["adj_close"].values.reshape(-1,1)).\
           reshape(len(df1),)
        
        joblib.dump(scaler, f"drive/My Drive/stock_pred/scalers/{company}_close.save") 
        
        print("scaler saved for {}".format(company))
        df2["adj_close"]=scaler.transform(df2["adj_close"].values.reshape(-1,1)).\
           reshape(len(df2),)
        
        df1["future"]=scaler.transform(df1["future"].values.reshape(-1,1)).\
           reshape(len(df1),)
        
        df2["future"]=scaler.transform(df2["future"].values.reshape(-1,1)).\
           reshape(len(df2),)
        
        
        scaler.fit(df1["adj_volume"].values.reshape(-1,1))
        df1["adj_volume"]=scaler.transform(df1["adj_volume"].values.reshape(-1,1)).\
           reshape(len(df1),)
        
        df2["adj_volume"]=scaler.transform(df2["adj_volume"].values.reshape(-1,1)).\
           reshape(len(df2),)
        
        joblib.dump(scaler, f"drive/My Drive/stock_pred/scalers/{company}_volume.save") 
        
        scaler.fit(df1["Index Value"].values.reshape(-1,1))
        df1["Index Value"]=scaler.transform(df1["Index Value"].values.reshape(-1,1)).\
           reshape(len(df1),)
        
        df2["Index Value"]=scaler.transform(df2["Index Value"].values.reshape(-1,1)).\
           reshape(len(df2),)
        
        joblib.dump(scaler, f"drive/My Drive/stock_pred/scalers/{company}_index.save") 

        
        df1.dropna(inplace=True) 
        df2.dropna(inplace=True)
        sequential_data_1 = [] 
        sequential_data_2 = [] 
        prev_days_1 = deque(maxlen=SEQ_LEN) 
        prev_days_2 = deque(maxlen=SEQ_LEN) 
    
        for i in df1.values: 
            prev_days_1.append([n for n in i[:-1]]) 
            if len(prev_days_1) == SEQ_LEN:  
                sequential_data_1.append([np.array(prev_days_1), i[-1]])
        
        random.shuffle(sequential_data_1) 
        
        for i in df2.values: 
            prev_days_2.append([n for n in i[:-1]]) 
            if len(prev_days_2) == SEQ_LEN:  
                sequential_data_2.append([np.array(prev_days_2), i[-1]])        
         
        random.shuffle(sequential_data_2) 
        
        x_t=[]
        y_t=[]
        x_v=[]
        y_v=[]
        
        for seq,target in sequential_data_1:
            x_t.append(seq)
            y_t.append(target)
            
        for seq,target in sequential_data_2:
            x_v.append(seq)
            y_v.append(target) 
            
        return np.array(x_t),y_t,np.array(x_v),y_v    

In [0]:
#MODEL
def make_model(train_x):
    model = Sequential()
    #model.add(TimeDistributed(Dense(1024, activation='relu')))
    #model.add(Dropout(0.2))
    #model.add(BatchNormalization())
    model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
    #(timesteps*data_dim )
    model.add(Dropout(0.2))
    model.add(BatchNormalization())  #normalizes activation outputs

    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(1, activation='linear'))
    
    return model
    
    

In [0]:
print(company[49])

INTC


In [0]:
for i in range(49,len(company)):
    RATIO_TO_PREDICT=company[i]
    main_df=pd.DataFrame()
    main_df=pd.read_csv(f"drive/My Drive/stock_pred/data/{company[i]}.csv")
    
    if len(main_df)==0:
        continue
    
    main_df.set_index("date",inplace=True)
    main_df=main_df[["adj_close","adj_volume"]]
    #print(main_df.head())
    main_df=main_df.join(df)
    main_df["future"]=main_df["adj_close"].shift(FUTURE_PERIOD_PREDICT)
    #main_df=main_df.iloc[::-1]
    #print(main_df.head())
    validation_df=main_df[:][-int(0.03*len(main_df)):]
    #print(len(validation_df))
    main_df=main_df[:][:int(0.97*len(main_df))]
    print(len(main_df))
    train_x,train_y,validation_x,validation_y=preprocess_df(main_df,validation_df)
    #print(len(train_y))
    model=make_model(train_x)
    
    opt = keras.optimizers.Adam(lr=0.001, decay=1e-6)
    
    model.compile(
    loss='mean_squared_error',
    optimizer=opt,
    metrics=['accuracy']
    )
    #train_t_y=np.array(train_t_y)
    #validation_t_y=np.array(validation_t_y)
    # Train model
    history = model.fit(
        train_x,train_y,
        batch_size=32,
        epochs=50,
        validation_data=(validation_x,validation_y)
        #callbacks=[tensorboard, checkpoint],
    )

    # Score model
    #score = model.evaluate(validation_x, validation_t_y, verbose=0)
    #print('Test loss:', score[0])
    #print('Test accuracy:', score[1])
    # Save model
    model.save(f"drive/My Drive/stock_pred/models/{company[i]}.h5")
    print(company[i])
    print("model saved")
    del model
    
    

3706
Train on 3694 samples, validate on 105 samples
Epoch 1/50
3694/3694 [==============================] - 10s 3ms/step - loss: 0.3054 - acc: 5.4142e-04 - val_loss: 0.1084 - val_acc: 0.0000e+00
Epoch 2/50
3694/3694 [==============================] - 8s 2ms/step - loss: 0.0920 - acc: 2.7071e-04 - val_loss: 0.0100 - val_acc: 0.0000e+00
Epoch 3/50
3694/3694 [==============================] - 8s 2ms/step - loss: 0.0487 - acc: 5.4142e-04 - val_loss: 0.0071 - val_acc: 0.0000e+00
Epoch 4/50
3694/3694 [==============================] - 8s 2ms/step - loss: 0.0290 - acc: 2.7071e-04 - val_loss: 0.0025 - val_acc: 0.0000e+00
Epoch 5/50
3694/3694 [==============================] - 8s 2ms/step - loss: 0.0213 - acc: 5.4142e-04 - val_loss: 3.5695e-04 - val_acc: 0.0000e+00
Epoch 6/50
3694/3694 [==============================] - 8s 2ms/step - loss: 0.0155 - acc: 5.4142e-04 - val_loss: 0.0027 - val_acc: 0.0000e+00
Epoch 7/50
3694/3694 [==============================] - 8s 2ms/step - loss: 0.0143 - acc: 5

In [0]:
for i in range(0,len(company)):
    RATIO_TO_PREDICT=company[i]
    main_df=pd.DataFrame()
    main_df=pd.read_csv(f"drive/My Drive/stock_pred/data/{company[i]}.csv")
    
    if len(main_df)==0:
        continue
    
    main_df.set_index("date",inplace=True)
    main_df=main_df[["adj_close","adj_volume"]]
    #print(main_df.head())
    main_df=main_df.join(df)
    main_df["future"]=main_df["adj_close"].shift(FUTURE_PERIOD_PREDICT)
    #main_df=main_df.iloc[::-1]
    #print(main_df.head())
    validation_df=main_df[:][-int(0.03*len(main_df)):]
    #print(len(validation_df))
    main_df=main_df[:][:int(0.97*len(main_df))]
    #print(len(main_df))
    train_x,train_y,validation_x,validation_y=preprocess_df(main_df,validation_df,company[i])

scaler saved for AAL
scaler saved for AAPL
scaler saved for ADBE
scaler saved for ADI
scaler saved for ADP
scaler saved for ADSK
scaler saved for ALGN
scaler saved for ALXN
scaler saved for AMAT
scaler saved for AMGN
scaler saved for AMZN
scaler saved for ATVI
scaler saved for AVGO
scaler saved for BIDU
scaler saved for BIIB
scaler saved for BMRN
scaler saved for CDNS
scaler saved for CELG
scaler saved for CERN
scaler saved for CHKP
scaler saved for CHTR
scaler saved for CTAS
scaler saved for CSCO
scaler saved for CTXS
scaler saved for CMCSA
scaler saved for COST
scaler saved for CSX
scaler saved for CTSH
scaler saved for DISH
scaler saved for DLTR
scaler saved for EA
scaler saved for EBAY
scaler saved for ESRX
scaler saved for EXPE
scaler saved for FAST
scaler saved for FB
scaler saved for FISV
scaler saved for FOX
scaler saved for FOXA
scaler saved for GILD
scaler saved for GOOG
scaler saved for GOOGL
scaler saved for HAS
scaler saved for HSIC
scaler saved for HOLX
scaler saved for I

In [0]:
def take_input(c,v,i,comp):
    c=np.array(c)
    scaler0 = joblib.load(f"drive/My Drive/stock_pred/scalers/{comp}_close.save") 
    c=scaler0.transform(c.reshape(-1,1)).reshape(len(c),)
    v=np.array(v)
    scaler = joblib.load(f"drive/My Drive/stock_pred/scalers/{comp}_volume.save") 
    v=scaler.transform(v.reshape(-1,1)).reshape(len(v),)
    i=np.array(i)
    scaler = joblib.load(f"drive/My Drive/stock_pred/scalers/{comp}_index.save") 
    i=scaler.transform(i.reshape(-1,1)).reshape(len(i),)
    inp=np.array([])
    for j in range(0,10):
        temp=[]
        temp.append(c[j])
        temp.append(v[j])
        temp.append(i[j])
        temp=np.array(temp)
        inp=np.concatenate((inp,temp))
        
      
    
    inp=inp.reshape(1,10,3)  
    print(inp.shape)
    model=load_model(f"drive/My Drive/stock_pred/models/{comp}.h5")
    ans=scaler0.inverse_transform(model.predict(inp))
    #print(inp)
    return ans
    

In [0]:
c=[56.5,56.23,55.28,55.17,55.4,54.62,55.32,54.09,52.31,51.86]
v=[3177925,4343055,4262349,2964323,4455977,3743641,2497070,4785030,\
  4820379,3843474]
i=[6750.54,6811.04,6881,6913,6929,6966,7101,7131,7046,7040,7030]

print(take_input(c,v,i,"AAL"))
#scaler.inverse_transform(scaled)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.19.2 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.19.2 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.19.2 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


(1, 10, 3)
[[48.773895]]


In [0]:
import numpy as np

vec = np.zeros(10).reshape((1, 10))
print((vec)+[1,2,3,4,5,6,5,4,3,6])

[[1. 2. 3. 4. 5. 6. 5. 4. 3. 6.]]
